Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50098)
clean_all_dbs(speos.client)

Create project

In [ ]:
p = script.Project(speos=speos)
print(p)

# Create
## Root part

In [ ]:
root_part = p.create_root_part().commit()
print(root_part)

## Create bodies in root part.

In [ ]:
body_b1 = root_part.create_body(name="TheBodyB1").commit()
body_b2 = root_part.create_body(name="TheBodyB2").commit()
print(root_part)

### Create faces in a body

In [ ]:
face_b1_f1 = body_b1.create_face(name="TheFaceF1").set_vertices([0,0,0,1,0,0,0,1,0])\
                                                  .set_facets([0,1,2])\
                                                  .set_normals([0,0,1,0,0,1,0,0,1])\
                                                  .commit()
print(root_part)

## Create sub parts in root part.

In [ ]:
sub_part1 = root_part.create_sub_part(name="TheSubPartSP1").set_axis_system(axis_system=[5,5,5,1,0,0,0,1,0,0,0,1]).commit()
print(root_part)

### Create bodies in a sub part

In [ ]:
body_sp1_b1 = sub_part1.create_body(name="TheBodySP1_B1").commit()
print(root_part)

### Create faces in a body

In [ ]:
face_sp1_b1_f1 = body_sp1_b1.create_face(name="TheFaceSP1_B1_F1").set_vertices([0,1,0,0,2,0,1,2,0])\
                                                                 .set_facets([0,1,2])\
                                                                 .set_normals([0,0,1,0,0,1,0,0,1])\
                                                                 .commit()
print(root_part)

## Create sub parts in sub part

In [ ]:
sub_part11 = sub_part1.create_sub_part(name="TheSubPartSP11").set_axis_system([1,1,1,1,0,0,0,1,0,0,0,1]).commit()
print(root_part)

# Find
## The root part

In [ ]:
features = p.find(name="", feature_type=script.Part)
print(features[0])

## A Specific body in the root part

In [ ]:
features = p.find(name="TheBodyB1", feature_type=script.Body)
print(features[0])

## A face in a body of the root part

In [ ]:
features = p.find(name="TheBodyB1/TheFaceF1", feature_type=script.Face)
print(features[0])

## A Sub part in the root part

In [ ]:
features = p.find(name="TheSubPartSP1", feature_type=script.Part.SubPart)
print(features[0])

## A Body in a sub part of the root part

In [ ]:
features = p.find(name="TheSubPartSP1/TheBodySP1_B1", feature_type=script.Body)
print(features[0])

## A Face in a body of a sub part of the root part

In [ ]:
features = p.find(name="TheSubPartSP1/TheBodySP1_B1/TheFaceSP1_B1_F1", feature_type=script.Face)
print(features[0])

# Find with regex
## All bodies at first level

In [ ]:
features = p.find(name=".*", name_regex=True, feature_type=script.Body)
for feat in features:
    print(feat._name)

## All faces of a specific body

In [ ]:
features = p.find(name="TheBodyB1/.*", name_regex=True, feature_type=script.Face)
for feat in features:
    print(feat._name)

## All features at a level
If you want to retrieve several kind of geometry features, give feature_type=script.Part

In [ ]:
features = p.find(name=".*", name_regex=True, feature_type=script.Part)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

In [ ]:
features = p.find(name=".*/.*", name_regex=True, feature_type=script.Part)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

In [ ]:
features = p.find(name=".*/.*/.*", name_regex=True, feature_type=script.Part)
for feat in features:
    print(str(type(feat)) + " : name=" + feat._name)

# Delete

In [ ]:
root_part.delete()
print(root_part)